In [1]:
import os
import xml.etree.ElementTree as ET
import psycopg2

# Define the PostgreSQL connection parameters
db_params = {
    "host": "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com",
    "database": "CSPODB",
    "port": 5432,
    "user": "cspoms",
    "password": "cspoms",
    "sslmode": "verify-ca",
    "sslcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-cert.pem",
    "sslkey": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-key.pem",
    "sslrootcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\server-ca.pem"
}

# Define the root directory path
root_directory = r'D:\W\RELEX\RELEX\Future_List_Cost\Inbound_files'

# Specify the file name
# file_name = 'future.xml'
file_name = f'FUTRLIST.IN07.20231206100039 (Oracle File)'

# Join the root directory path and the file name
file_path = os.path.join(root_directory, file_name)

# Open the file and parse each line
with open(file_path, 'r', encoding='utf-8') as file:
    # Establish a database connection and execute the INSERT statement
    with psycopg2.connect(**db_params) as conn:
        with conn.cursor() as cursor:
            for line in file:
                try:
                    # Parse each line as XML
                    root = ET.fromstring(line.strip())

                    # Access the elements within each <LIST_COST_MAINT>
                    BICEPSITEM = root.find('BICEPSITEM').text
                    FIRSTORDERDATE = root.find('FIRSTORDERDATE').text
                    LASTORDERDATE = root.find('LASTORDERDATE').text
                    ITEMLISTFUTURE = root.find('ITEMLISTFUTURE').text
                    DATESELLEFF = root.find('DATESELLEFF').text
                    DEALMANUFACTURENUMBER = root.find('DEALMANUFACTURENUMBER').text
                    LISTCOSTADD = root.find('LISTCOSTADD').text
                    TRANSACTIONID = root.find('TRANSACTIONID').text

                    # Execute the INSERT statement for each line
                    cursor.execute("""
                        INSERT INTO LIST_COST_MAINT_STG (
                            BICEPSITEM, FIRSTORDERDATE, LASTORDERDATE,
                            ITEMLISTFUTURE, DATESELLEFF, DEALMANUFACTURENUMBER,
                            LISTCOSTADD, TRANSACTIONID
                        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                    """, (
                        BICEPSITEM, FIRSTORDERDATE, LASTORDERDATE,
                        ITEMLISTFUTURE, DATESELLEFF, DEALMANUFACTURENUMBER,
                        LISTCOSTADD, TRANSACTIONID
                    ))

                except ET.ParseError as e:
                    print(f"Error parsing line: {line.strip()}")
                    print(f"ParseError: {e}")

# Commit the changes to the database
conn.commit()


Error parsing line: <LIST_COST_MAINT><BICEPSITEM>250473199</BICEPSITEM> <FIRSTORDERDATE>120723</FIRSTORDERDATE> <LASTORDERDATE>010624</LASTORDERDATE> <ITEMLISTFUTURE>00046.1800</ITEMLISTFUTURE> <DATESELLEFF>010724</DATESELLEFF> <DEALMANUFACTURENUMBER>120730106ORA4618</DEALMANUFACTURENUMBER> <LISTCOSTADD></LISTCOSTADD>       <TRANSACTIONID>L120623000000000000001<TRANSACTIONID> </LIST_COST_MAINT>
ParseError: mismatched tag: line 1, column 361
Error parsing line: <LIST_COST_MAINT><BICEPSITEM>250494237</BICEPSITEM> <FIRSTORDERDATE>120723</FIRSTORDERDATE> <LASTORDERDATE>010624</LASTORDERDATE> <ITEMLISTFUTURE>00024.2600</ITEMLISTFUTURE> <DATESELLEFF>010724</DATESELLEFF> <DEALMANUFACTURENUMBER>120730106ORA2426</DEALMANUFACTURENUMBER> <LISTCOSTADD></LISTCOSTADD>       <TRANSACTIONID>L120623000000000000002<TRANSACTIONID> </LIST_COST_MAINT>
ParseError: mismatched tag: line 1, column 361
Error parsing line: <LIST_COST_MAINT><BICEPSITEM>250091739</BICEPSITEM> <FIRSTORDERDATE>030824</FIRSTORDERDATE

In [4]:
import os
import re
import psycopg2

# ... (Same PostgreSQL connection parameters and root_directory)

# Specify the file name
file_name = f'FUTRLIST.IN07.20231206100039 (Oracle File)'

# Join the root directory path and the file name
file_path = os.path.join(root_directory, file_name)

# Establish a database connection
with psycopg2.connect(**db_params) as conn:
    with conn.cursor() as cursor:
        # Open the file and insert each line into the database
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                try:
                    # Use regular expression to split values based on <tag>value</tag>
                    values = re.findall(r'<[^>]+>([^<]+)<[^>]+>', line)

                    # Access the values based on their positions
                    BICEPSITEM = values[0]
                    FIRSTORDERDATE = values[1]
                    LASTORDERDATE = values[2]
                    ITEMLISTFUTURE = values[3]
                    DATESELLEFF = values[4]
                    DEALMANUFACTURENUMBER = values[5]
                    LISTCOSTADD = values[6]
                    TRANSACTIONID = values[7]

                    # Execute the INSERT statement for each line
                    cursor.execute("""
                        INSERT INTO xxpo_future_list_cost_master (
                            fac_item_number, first_order_date, last_order_date,
                            future_cost, sell_effective_date, mfg_deal_num,
                            list_cost_type, deal_type, transaction_id
                        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
                    """, (
                        BICEPSITEM, FIRSTORDERDATE, LASTORDERDATE,
                        ITEMLISTFUTURE, DATESELLEFF, DEALMANUFACTURENUMBER,
                        LISTCOSTADD, "FLC", TRANSACTIONID
                    ))

                except IndexError as e:
                    print(f"Error processing line: {line.strip()}")
                    print(f"IndexError: {e}")

        # Commit the changes to the database
        conn.commit()



In [11]:
import os
import re
import psycopg2
from datetime import datetime

db_params = {
    "host": "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com",
    "database": "CSPODB",
    "port": 5432,
    "user": "cspoms",
    "password": "cspoms",
    "sslmode": "verify-ca",
    "sslcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-cert.pem",
    "sslkey": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-key.pem",
    "sslrootcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\server-ca.pem"
}

# Establish a database connection
with psycopg2.connect(**db_params) as post_conn:
    with post_conn.cursor() as post_cur:
        # Get the nextval for request_id
        nextval_request_id = "SELECT nextval('cspoms.xxpo_cs_relex_run_id_seq')"
        post_cur.execute(nextval_request_id)
        request_id = post_cur.fetchone()[0]
        post_conn.commit()
        print(f"Request ID: {request_id}")

        # Get the current timestamp
        current_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"Current Timestamp: {current_timestamp}")

# Specify the file name
file_name = f'FUTRLIST.IN07.20231206100039 (Oracle File)'

# Join the root directory path and the file name
file_path = os.path.join(root_directory, file_name)

# Establish a database connection
with psycopg2.connect(**db_params) as conn:
    with conn.cursor() as cursor:
        # Open the file and insert each line into the database
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                try:
                    # Use regular expression to find key-value pairs
                    key_value_pairs = re.findall(r'<([^>]+)>([^<]+)<\/\1>', line)

                    # Convert key-value pairs to a dictionary
                    values_dict = dict(key_value_pairs)

                    # Access the values based on their keys
                    BICEPSITEM = values_dict.get('BICEPSITEM', '')
                    FIRSTORDERDATE = values_dict.get('FIRSTORDERDATE', '')
                    LASTORDERDATE = values_dict.get('LASTORDERDATE', '')
                    ITEMLISTFUTURE = values_dict.get('ITEMLISTFUTURE', '')
                    DATESELLEFF = values_dict.get('DATESELLEFF', '')
                    DEALMANUFACTURENUMBER = values_dict.get('DEALMANUFACTURENUMBER', '')
                    LISTCOSTADD = values_dict.get('LISTCOSTADD', '')
                    TRANSACTIONID = values_dict.get('TRANSACTIONID', '')

                    # Execute the INSERT statement for each line into LIST_COST_MAINT_STG
                    cursor.execute("""INSERT INTO LIST_COST_MAINT_STG (
                                        request_id, BICEPSITEM, FIRSTORDERDATE, LASTORDERDATE,
                                        ITEMLISTFUTURE, DATESELLEFF, DEALMANUFACTURENUMBER,
                                        LISTCOSTADD, TRANSACTIONID, CREATED_BY, LAST_UPDATED_BY,
                                        CREATION_DATE, LAST_UPDATE_DATE
                                    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                                   """, (
                                    request_id, BICEPSITEM, FIRSTORDERDATE, LASTORDERDATE,
                                    ITEMLISTFUTURE, DATESELLEFF, DEALMANUFACTURENUMBER,
                                    LISTCOSTADD, TRANSACTIONID, -1, -1, current_timestamp, current_timestamp
                                ))
#                     cursor.execute("""
#                         INSERT INTO LIST_COST_MAINT_STG (
#                             BICEPSITEM, FIRSTORDERDATE, LASTORDERDATE,
#                             ITEMLISTFUTURE, DATESELLEFF, DEALMANUFACTURENUMBER,
#                             LISTCOSTADD, TRANSACTIONID
#                         ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
#                     """, (
#                         BICEPSITEM, FIRSTORDERDATE, LASTORDERDATE,
#                         ITEMLISTFUTURE, DATESELLEFF, DEALMANUFACTURENUMBER,
#                         LISTCOSTADD, TRANSACTIONID
#                     ))

                except Exception as e:
                    print(f"Error processing line: {line.strip()}")
                    print(f"Error: {e}")

        # Commit the changes to the database
        conn.commit()


Request ID: 4
Current Timestamp: 2023-12-29 18:36:01


In [13]:
import os
import re
import psycopg2
from datetime import datetime

db_params = {
    "host": "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com",
    "database": "CSPODB",
    "port": 5432,
    "user": "cspoms",
    "password": "cspoms",
    "sslmode": "verify-ca",
    "sslcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-cert.pem",
    "sslkey": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-key.pem",
    "sslrootcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\server-ca.pem"
}

# Establish a database connection
with psycopg2.connect(**db_params) as post_conn:
    with post_conn.cursor() as post_cur:
        # Get the nextval for request_id
        nextval_request_id = "SELECT nextval('cspoms.xxpo_cs_relex_run_id_seq')"
        post_cur.execute(nextval_request_id)
        request_id = post_cur.fetchone()[0]
        post_conn.commit()
        print(f"Request ID: {request_id}")

        # Get the current timestamp
        current_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"Current Timestamp: {current_timestamp}")

# Specify the file name
file_name = f'FUTRLIST.IN07.20231206100039 (Oracle File)'

# Join the root directory path and the file name
file_path = os.path.join(root_directory, file_name)

# Establish a database connection
with psycopg2.connect(**db_params) as conn:
    with conn.cursor() as cursor:
        # Open the file and insert each line into the database
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                try:
                    # Extract values using regular expression
                    values = re.findall(r'<([^>]+)>([^<]+)<\/\1>', line)

                    # Handle TRANSACTIONID separately due to lack of closing tag
                    transactionid_match = re.search(r'<TRANSACTIONID>([^<]+)<', line)
                    TRANSACTIONID = transactionid_match.group(1) if transactionid_match else ''

                    # Convert other key-value pairs to a dictionary
                    values_dict = dict(values)

                    # Access the values based on their keys
                    BICEPSITEM = values_dict.get('BICEPSITEM', '')
                    FIRSTORDERDATE = values_dict.get('FIRSTORDERDATE', '')
                    LASTORDERDATE = values_dict.get('LASTORDERDATE', '')
                    ITEMLISTFUTURE = values_dict.get('ITEMLISTFUTURE', '')
                    DATESELLEFF = values_dict.get('DATESELLEFF', '')
                    DEALMANUFACTURENUMBER = values_dict.get('DEALMANUFACTURENUMBER', '')
                    LISTCOSTADD = values_dict.get('LISTCOSTADD', '')

                    # Execute the INSERT statement for each line into LIST_COST_MAINT_STG
                    cursor.execute("""INSERT INTO LIST_COST_MAINT_STG (
                                        request_id, BICEPSITEM, FIRSTORDERDATE, LASTORDERDATE,
                                        ITEMLISTFUTURE, DATESELLEFF, DEALMANUFACTURENUMBER,
                                        LISTCOSTADD, TRANSACTIONID, CREATED_BY, LAST_UPDATED_BY,
                                        CREATION_DATE, LAST_UPDATE_DATE
                                    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                                   """, (
                                    request_id, BICEPSITEM, FIRSTORDERDATE, LASTORDERDATE,
                                    ITEMLISTFUTURE, DATESELLEFF, DEALMANUFACTURENUMBER,
                                    LISTCOSTADD, TRANSACTIONID, -1, -1, current_timestamp, current_timestamp
                                ))
                    
                                # Get the number of rows inserted
                    num_rows_inserted = cursor.rowcount

                    # Print the result
                    print(f"Inserted {num_rows_inserted} rows.")

                except Exception as e:
                    print(f"Error processing line: {line.strip()}")
                    print(f"Error: {e}")

        # Commit the changes to the database
        conn.commit()


Request ID: 6
Current Timestamp: 2023-12-29 18:41:49
Inserted 1 rows.
Inserted 1 rows.
Inserted 1 rows.
Inserted 1 rows.
Inserted 1 rows.
Inserted 1 rows.
Inserted 1 rows.
Inserted 1 rows.


In [15]:
import os
import re
import psycopg2
from datetime import datetime

# ... (Same PostgreSQL connection parameters and root_directory)

# Establish a database connection
with psycopg2.connect(**db_params) as post_conn:
    with post_conn.cursor() as post_cur:
        # Get the nextval for request_id
        nextval_request_id = "SELECT nextval('cspoms.xxpo_cs_relex_run_id_seq')"
        post_cur.execute(nextval_request_id)
        request_id = post_cur.fetchone()[0]
        post_conn.commit()
        print(f"Request ID: {request_id}")

        # Get the current timestamp
        current_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"Current Timestamp: {current_timestamp}")

# Specify the file name
file_name = f'FUTRLIST.IN07.20231206100039 (Oracle File)'

# Join the root directory path and the file name
file_path = os.path.join(root_directory, file_name)

# Establish a database connection
with psycopg2.connect(**db_params) as conn:
    with conn.cursor() as cursor:
        # Open the file and insert each line into the database
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                try:
                    # Extract values using regular expression
                    values = re.findall(r'<([^>]+)>([^<]+)<\/\1>', line)

                    # Handle TRANSACTIONID separately due to lack of closing tag
                    transactionid_match = re.search(r'<TRANSACTIONID>([^<]+)<', line)
                    TRANSACTIONID = transactionid_match.group(1) if transactionid_match else ''

                    # Convert other key-value pairs to a dictionary
                    values_dict = dict(values)

                    # Access the values based on their keys
                    BICEPSITEM = values_dict.get('BICEPSITEM', '')
                    FIRSTORDERDATE = values_dict.get('FIRSTORDERDATE', '')
                    LASTORDERDATE = values_dict.get('LASTORDERDATE', '')
                    ITEMLISTFUTURE = values_dict.get('ITEMLISTFUTURE', '')
                    DATESELLEFF = values_dict.get('DATESELLEFF', '')
                    DEALMANUFACTURENUMBER = values_dict.get('DEALMANUFACTURENUMBER', '')
                    LISTCOSTADD = values_dict.get('LISTCOSTADD', '')

                    # Execute the INSERT statement for each line into LIST_COST_MAINT_STG
                    cursor.execute("""INSERT INTO LIST_COST_MAINT_STG (
                                        request_id, BICEPSITEM, FIRSTORDERDATE, LASTORDERDATE,
                                        ITEMLISTFUTURE, DATESELLEFF, DEALMANUFACTURENUMBER,
                                        LISTCOSTADD, TRANSACTIONID, CREATED_BY, LAST_UPDATED_BY,
                                        CREATION_DATE, LAST_UPDATE_DATE
                                    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                                   """, (
                                    request_id, BICEPSITEM, FIRSTORDERDATE, LASTORDERDATE,
                                    ITEMLISTFUTURE, DATESELLEFF, DEALMANUFACTURENUMBER,
                                    LISTCOSTADD, TRANSACTIONID, -1, -1, current_timestamp, current_timestamp
                                ))
                    
                    

                except Exception as e:
                    print(f"Error processing line: {line.strip()}")
                    print(f"Error: {e}")

        # Commit the changes to the database
        conn.commit()


print(f"Records inserted to staging table with request_id : {request_id}")

Request ID: 8
Current Timestamp: 2023-12-29 18:45:06
Records inserted to staging table with request_id : 8


In [24]:
import os
import re
import psycopg2
from datetime import datetime
import pandas as pd

db_params = {
    "host": "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com",
    "database": "CSPODB",
    "port": 5432,
    "user": "cspoms",
    "password": "cspoms",
    "sslmode": "verify-ca",
    "sslcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-cert.pem",
    "sslkey": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-key.pem",
    "sslrootcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\server-ca.pem"
}

# Function to process a file
def process_file(file_path, request_id, current_timestamp, cursor):
    print(f"Processing file: {file_path}")
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                # Extract values using regular expression
                values = re.findall(r'<([^>]+)>([^<]+)<\/\1>', line)

                # Handle TRANSACTIONID separately due to lack of closing tag
                transactionid_match = re.search(r'<TRANSACTIONID>([^<]+)<', line)
                TRANSACTIONID = transactionid_match.group(1) if transactionid_match else ''

                # Convert other key-value pairs to a dictionary
                values_dict = dict(values)

                # Access the values based on their keys
                BICEPSITEM = values_dict.get('BICEPSITEM', '')
                FIRSTORDERDATE = values_dict.get('FIRSTORDERDATE', '')
                LASTORDERDATE = values_dict.get('LASTORDERDATE', '')
                ITEMLISTFUTURE = values_dict.get('ITEMLISTFUTURE', '')
                DATESELLEFF = values_dict.get('DATESELLEFF', '')
                DEALMANUFACTURENUMBER = values_dict.get('DEALMANUFACTURENUMBER', '')
                LISTCOSTADD = values_dict.get('LISTCOSTADD', '')
                print(f"Inserting values into database for line: {line.strip()}")

                # Execute the INSERT statement for each line into LIST_COST_MAINT_STG
                cursor.execute("""INSERT INTO LIST_COST_MAINT_STG (
                                    request_id, BICEPSITEM, FIRSTORDERDATE, LASTORDERDATE,
                                    ITEMLISTFUTURE, DATESELLEFF, DEALMANUFACTURENUMBER,
                                    LISTCOSTADD, TRANSACTIONID, CREATED_BY, LAST_UPDATED_BY,
                                    CREATION_DATE, LAST_UPDATE_DATE
                                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                               """, (
                                request_id, BICEPSITEM, FIRSTORDERDATE, LASTORDERDATE,
                                ITEMLISTFUTURE, DATESELLEFF, DEALMANUFACTURENUMBER,
                                LISTCOSTADD, TRANSACTIONID, -1, -1, current_timestamp, current_timestamp
                            ))

            except Exception as e:
                print(f"Error processing line: {line.strip()}")
                print(f"Error: {e}")

# Establish a database connection
with psycopg2.connect(**db_params) as post_conn:
    with post_conn.cursor() as post_cur:
        # Get the nextval for request_id
        nextval_request_id = "SELECT nextval('cspoms.xxpo_cs_relex_run_id_seq')"
        post_cur.execute(nextval_request_id)
        request_id = post_cur.fetchone()[0]
        post_conn.commit()
        print(f"Request ID: {request_id}")

        # Get the current timestamp
        current_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"Current Timestamp: {current_timestamp}")

        # Specify the file name pattern
        file_name_pattern = 'FUTRLIST.IN07'

        # Iterate over files in the directory
        for file_name in os.listdir(root_directory):
            if file_name_pattern in file_name:
                # Join the root directory path and the file name
                file_path = os.path.join(root_directory, file_name)

                # Establish a database connection
                with psycopg2.connect(**db_params) as conn:
                    print('db_params :', db_params)
                    print('conn :',conn)
                    with conn.cursor() as cursor:
                        # Process the file
                        process_file(file_path, request_id, current_timestamp, cursor)
                        
                        # Commit the changes to the database
                        conn.commit()
         # Fetch the df for the request_id
        post_cur.execute(f'SELECT * FROM LIST_COST_MAINT_STG WHERE request_id = {request_id}')
        columns = [desc[0] for desc in post_cur.description]
        df = pd.DataFrame(post_cur.fetchall(), columns=columns)

print(f"Records inserted to staging table with request_id: {request_id}")
print("DataFrame for the inserted records:")
print(df)



Request ID: 15
Current Timestamp: 2024-01-02 18:29:15
db_params : {'host': 'gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com', 'database': 'CSPODB', 'port': 5432, 'user': 'cspoms', 'password': 'cspoms', 'sslmode': 'verify-ca', 'sslcert': 'D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-cert.pem', 'sslkey': 'D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-key.pem', 'sslrootcert': 'D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\server-ca.pem'}
conn : <connection object at 0x000002151AC416A0; dsn: 'user=cspoms password=xxx dbname=CSPODB host=gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com port=5432 sslmode=verify-ca sslcert=D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-cert.pem sslkey=D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-key.pem sslrootcert=D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\server-ca.pem', closed: 0>
Processing file: D:\W\RELEX\RELEX\Future_List_Cost\Inbound_files\FUTRLIST.IN07.20231205190925690 (MF File)
Inserting values into database for line: <LIST_COST_MAINT><BICEPSITEM>250473199</BICEPSITEM> <FIR

Records inserted to staging table with request_id: 15
DataFrame for the inserted records:
   bicepsitem firstorderdate lastorderdate itemlistfuture dateselleff  \
0   250473199         120723        010624     00046.1800      010724   
1   250494237         120723        010624     00024.2600      010724   
2   250091739         030824        040724     00014.2000      040824   
3   250952159         030824        040724     00013.5200      040824   
4   250494237         030824        040724     00022.6500      040824   
5   250473199         030824        040724     00040.9300      040824   
6   251582038         010524        020424     00187.2000      020524   
7   251580418         010524        020424     00019.9200      020524   
8   250473199         120723        010624     00046.1800      010724   
9   250494237         120723        010624     00024.2600      010724   
10  250091739         030824        040724     00014.2000      040824   
11  250952159         030824      

In [26]:
def replace_second_occurrence(input_str, search_str, replace_str):
    # Find the first occurrence of the search string
    first_occurrence = input_str.find(search_str)

    # If the search string is found
    if first_occurrence != -1:
        # Find the second occurrence of the search string starting from the position after the first occurrence
        second_occurrence = input_str.find(search_str, first_occurrence + 1)

        # If the second occurrence is found
        if second_occurrence != -1:
            # Replace the second occurrence with the replace string
            output_str = input_str[:second_occurrence] + replace_str + input_str[second_occurrence + len(search_str):]

            return output_str

    # If either the first or second occurrence is not found, return the original string
    return input_str

# Example usage
input_string = "<LIST_COST_MAINT><BICEPSITEM>250473199</BICEPSITEM> <FIRSTORDERDATE>120723</FIRSTORDERDATE> <LASTORDERDATE>010624</LASTORDERDATE> <ITEMLISTFUTURE>00046.1800</ITEMLISTFUTURE> <DATESELLEFF>010724</DATESELLEFF> <DEALMANUFACTURENUMBER>120730106ORA4618</DEALMANUFACTURENUMBER> <LISTCOSTADD></LISTCOSTADD>       <TRANSACTIONID>L120623000000000000001<TRANSACTIONID> </LIST_COST_MAINT>                                                                                                                            "
search_string = "<TRANSACTIONID>"
replace_string = "</TRANSACTIONID>"

result = replace_second_occurrence(input_string, search_string, replace_string)
print(result)

<LIST_COST_MAINT><BICEPSITEM>250473199</BICEPSITEM> <FIRSTORDERDATE>120723</FIRSTORDERDATE> <LASTORDERDATE>010624</LASTORDERDATE> <ITEMLISTFUTURE>00046.1800</ITEMLISTFUTURE> <DATESELLEFF>010724</DATESELLEFF> <DEALMANUFACTURENUMBER>120730106ORA4618</DEALMANUFACTURENUMBER> <LISTCOSTADD></LISTCOSTADD>       <TRANSACTIONID>L120623000000000000001</TRANSACTIONID> </LIST_COST_MAINT>                                                                                                                            
